# Library importing

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# data pre-proccessing

## data reading

In [2]:
data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

In [3]:
data.head(3)

,item_id,store_id,d,sales,date,wm_yr_wk,wday,event_name_1,event_type_1,event_name_2,event_type_2,snap,price_in_dollars
0,FOODS_1_001,CA_1,d_1,3,2011-01-29,11101,1,No event,No event,No event,No event,0,6.0
1,FOODS_1_001,CA_1,d_10,0,2011-02-07,11102,3,No event,No event,No event,No event,1,0.0
2,FOODS_1_001,CA_1,d_100,0,2011-05-08,11115,2,Mother's day,Cultural,No event,No event,1,0.0


## feature engineering

In [4]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [5]:
data['price']= data['price_in_dollars']/data['sales']
data.fillna(0,inplace=True)

In [6]:
data['wm_yr_wk']=data['wm_yr_wk']%100

In [7]:
data.drop(columns = ['d'],inplace = True)
data.drop(columns=['date'],inplace = True)
data.drop(columns = ['price_in_dollars'],inplace = True)

In [8]:
data.to_csv('/kaggle/working/preprocessing')

## Encoding

In [9]:
data['event_name_1'].nunique()

31

In [10]:
categorical_cols = ['item_id','store_id','event_name_1','event_name_2','event_type_1','event_type_2']
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "price"]

In [11]:
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value")),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])

In [12]:
preprocessor

,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,dtype,<class 'numpy.float64'>
,handle_unknown,'use_encoded_value'


## Data spliting

In [13]:
y=data['sales']
x=data.drop('sales',axis = 1)

# Model